In [2]:
# import the necessary packages
from sklearn.cluster import KMeans
from collections import Counter
import pandas as pd
import numpy as np
import urllib
import cv2
import time
import os
from os import listdir
from os.path import isfile, join
from math import sqrt

import colour

In [68]:
def jarak(r1,g1,b1, r2,g2,b2):
    a= r1 - r2
    b= g1 - g2
    c= b2 - b2
    d = (a**2)+(b**2)+(c**2)
    return d
# function to replace char in a string
def clean_sw(string, x,y):
    try:
        x = str(string).replace(x,y)
    except:
        x = ""
    return x
# METHOD #1: OpenCV, NumPy, and urllib
def url_to_image(url):
    # download the image, convert it to a NumPy array, and then read
    # it into OpenCV format
    resp = urllib.request.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # return the image
    return image,gray
# METHOD #2: get RGB
def get_dominant_color(image, k=5, l=3, image_processing_size = None):
    #resize image if new dims provided
    if image_processing_size is not None:
        image = cv2.resize(image, image_processing_size, 
                            interpolation = cv2.INTER_AREA)
    
    #reshape the image to be a list of pixels
    image = image.reshape((image.shape[0] * image.shape[1], 3))

    #cluster and assign labels to the pixels 
    clt = KMeans(n_clusters = k)
    labels = clt.fit_predict(image)

    #count labels to find most popular
    label_counts = Counter(labels)

    #subset out most popular centroid
    dominant_color = []
    for i in range(l):
        x = clt.cluster_centers_[label_counts.most_common(4)[i][0]]
        dominant_color.append(x)
    return dominant_color
# METHOD #2: get RGB

def get_dominant_black(image, k=5, image_processing_size = None):
    #resize image if new dims provided
    if image_processing_size is not None:
        image = cv2.resize(image, image_processing_size, 
                            interpolation = cv2.INTER_AREA)
    
    #reshape the image to be a list of pixels
    image = image.reshape((image.shape[0] * image.shape[1], 3))

    #cluster and assign labels to the pixels 
    clt = KMeans(n_clusters = k)
    labels = clt.fit_predict(image)

    #count labels to find most popular
    label_counts = Counter(labels)

    #subset out most popular centroid
    dominant_black = clt.cluster_centers_[label_counts.most_common(1)[0][0]]
    return dominant_black

In [64]:
# Iterate files in specific folder
cwd = os.getcwd()
files = []
for i in listdir(cwd + r"\\input"):
    if ".jpg" in i:
        files.append(i)
df_color = pd.read_csv('color_fix.csv', sep=';', index_col=0)
columns_order = ['file_name','top','parent_color','child_color','r','g','b','h','l','s','b_med','b_std','brightness','brightness_lab', 'hue','luminance']

In [78]:
bucket = []
for file in files:
    x = r'\\input'+ r"\\" + file
    img = cv2.imread(cwd+x)
    try:
        img_resized = cv2.resize(img,(32,32))
    except:
        pass
    try:
        # Get HLS
        img_grey = cv2.cvtColor(img_resized, cv2.COLOR_BGR2HLS)
        h,l,s = get_dominant_black(img_grey, k=5, image_processing_size=None)
        
    except:
        h,l,s = 999, 999, 999
    try:
        # Get Median and Std. Deviation
        med, std = np.median(img_grey),np.std(img_grey)
    except:
        med, std = 999, 999
    try:
        # Get RGB
        dominant_color = get_dominant_color(img_resized, k=5,l=3, image_processing_size = None)
        count = 1
        for a in dominant_color:
            
            df_color["jarak"] = np.vectorize(jarak)(df_color['r'].astype(int), df_color['g'].astype(int),df_color['b'].astype(int), int(a[0]), int(a[1]), int(a[2]))
            
            index_min = df_color.jarak.idxmin()
            parent_color = df_color.iloc[index_min].parent
            child_color = df_color.iloc[index_min].child
            luminance = df_color.iloc[index_min].luminance
            brightness = df_color.iloc[index_min].brightness
            hue = df_color.iloc[index_min].hue
            brightness_lab = df_color.iloc[index_min].brightness_lab
            
            bucket.append({'file_name':file, 'parent_color':parent_color,'child_color':child_color, 'top':count,
                        'r':a[0], 'g':a[1], 'b':a[2],
                          'h':h, 'l':l, 's':s,
                          'b_med':med, 'b_std':std,
                          'luminance':luminance,
                          'brightness':brightness,
                          'hue':hue,
                          'brightness_lab':brightness_lab})
            count += 1
    except:
        dominant_color = 999, 999, 999

start
ok
ok
ok


In [79]:
# Generate Data Frame
df = pd.DataFrame(bucket)
df = df[columns_order]

In [83]:
df.head()

,file_name,top,parent_color,child_color,r,g,b,h,l,s,b_med,b_std,brightness,brightness_lab,hue,luminance
0,1001704.jpg,1,Yellow_colors,Yellow,252.478000,253.262000,252.692000,101.031553,193.126214,94.684466,103.0,91.86313,170.000000,base,60.0,236.589
1,1001704.jpg,2,"Purple,_violet,_and_magenta_colors",Thistle,209.823899,190.896226,157.962264,101.031553,193.126214,94.684466,103.0,91.86313,207.666667,light,300.0,198.120
2,1001704.jpg,3,Gray_and_black_colors,Gainsboro,215.581197,216.316239,212.213675,101.031553,193.126214,94.684466,103.0,91.86313,220.000000,light,0.0,220.000
